<a href="https://colab.research.google.com/github/saiganeshreddy7/AI-Assisted-Cinematic-Storytelling-Software-for-Movie-Creation/blob/Main/TexttoVideoandEditing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/diffusers transformers accelerate


  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-m9jd07zb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-m9jd07zb
  Resolved https://github.com/huggingface/diffusers to commit 3bce0f3da1c0c13c5589cd97946ddbf58b8a9031
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 

#Story Generation

In [ ]:
from transformers import pipeline
story_gen = pipeline("text-generation", "pranavpsv/gpt2-genre-story-generator")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = "\"<BOS> <Nature> \" + \"Ducks in lake swimming \"" #@param {type:"string"}
generated_story = story_gen(prompt)
print(generated_story)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '"<BOS> <Nature> " + "Ducks in lake swimming "</ Nature>  A duck-dived and found itself in a small fishing town about to go fishing. It was the first time a family had been together in years. The'}]


#Video Generation

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
prompt = "Ducks in lake swimming" #@param {type:"string"}
video_frames = pipe(prompt, num_inference_steps=25, num_frames=200).frames

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
video_path = export_to_video(video_frames[0], output_video_path="Ducks.mp4", fps=30)

#Video Cutting/Trimming

In [ ]:
!pip install moviepy
!apt-get install imagemagick

In [ ]:
from moviepy.editor import VideoFileClip

def cut_video(input_path, start_time, end_time, output_folder):
    video_clip = VideoFileClip(input_path)
    cut_video_clip = video_clip.subclip(start_time, end_time)
    output_file_path = f"{output_folder}/output_cut_video.mp4"
    cut_video_clip.write_videofile(output_file_path, codec="libx264", audio_codec="aac")
    video_clip.close()
    cut_video_clip.close()

input_path = "/content/A.mp4" #@param {type:"string"}
start_time = 5 #@param {type:"number"}
end_time = 9 #@param {type:"number"}
output_folder = "/content/ouput" #@param {type:"string"}

cut_video(input_path, start_time, end_time, output_folder)


In [ ]:
from IPython.display import HTML, Video

video_path = "/content/ouput/output_cut_video.mp4" #@param {type:"string"}
Video(video_path, width=400, height=300, embed=True)




```
# This is formatted as code
```

#Concatenating Videos

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

def concatenate_videos(video_paths, output_path, output_extension):
    video_clips = [VideoFileClip(path) for path in video_paths]
    concatenated_clip = concatenate_videoclips(video_clips, method="compose")
    final_output_path = f"{output_path}.{output_extension}"
    concatenated_clip.write_videofile(final_output_path, codec="libx264", audio_codec="aac")

    for clip in video_clips:
        clip.close()
    concatenated_clip.close()


video_path1 = "/content/A.mp4"  #@param {type:"string"}
video_path2 = "/content/Y.mp4"  #@param {type:"string"}
output_path = "/content/ouput"  #@param {type:"string"}
output_extension = "mp4"  #@param {type:"string"}

concatenate_videos([video_path1, video_path2], output_path, output_extension)


In [ ]:
from IPython.display import HTML, Video

video_path = "/content/ouput.mp4" #@param {type:"string"}
Video(video_path, width=400, height=300, embed=True)


#Slow Video

In [ ]:
from moviepy.editor import VideoFileClip

def slow_down_video(input_path: str, start_time: int, end_time: int, output_folder: str, factor: float = 2.0):
    video_clip = VideoFileClip(input_path)
    cut_video_clip = video_clip.subclip(start_time, end_time)
    slowed_down_clip = cut_video_clip.speedx(factor)
    output_file_path = f"{output_folder}/output_slowed_video.mp4"
    slowed_down_clip.write_videofile(output_file_path, codec="libx264", audio_codec="aac")
    video_clip.close()
    cut_video_clip.close()
    slowed_down_clip.close()

input_path = "/content/Ducks.mp4" #@param {type:"string"}
start_time = 0 #@param {type:"number"}
end_time = 7 #@param {type:"number"}
output_folder = "/content/path" #@param {type:"string"}
slow_down_factor = 0.5  # This will slow down the video by half
slow_down_video(input_path, start_time, end_time, output_folder, factor=slow_down_factor)


Moviepy - Building video /content/path/output_slowed_video.mp4.
Moviepy - Writing video /content/path/output_slowed_video.mp4



t:  94%|█████████▍| 395/420 [00:02<00:00, 142.16it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/Ducks.mp4, 196608 bytes wanted but 0 bytes read,at frame 200/201, at time 6.67/6.67 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  98%|█████████▊| 410/420 [00:02<00:00, 123.56it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/Ducks.mp4, 196608 bytes wanted but 0 bytes read,at frame 205/201, at time 6.83/6.67 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/path/output_slowed_video.mp4


In [ ]:
from IPython.display import HTML, Video

video_path = "/content/path/output_slowed_video.mp4" #@param {type:"string"}
Video(video_path, width=400, height=300, embed=True)


In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

def add_audio_to_video(input_video_path: str, input_audio_path: str, output_folder: str):
    video_clip = VideoFileClip(input_video_path)
    audio_clip = AudioFileClip(input_audio_path)
    video_with_audio = video_clip.set_audio(audio_clip)
    output_file_path = f"{output_folder}/output_video_with_audio.mp4"
    video_with_audio.write_videofile(output_file_path, codec="libx264", audio_codec="aac")
    video_clip.close()
    audio_clip.close()
    video_with_audio.close()

input_video_path = "/content/Ducks.mp4" #@param {type:"string"}
input_audio_path = "/content/techno.wav" #@param {type:"string"}
output_folder = "/content/path" #@param {type:"string"}
add_audio_to_video(input_video_path, input_audio_path, output_folder)


Moviepy - Building video /content/path/output_video_with_audio.mp4.
MoviePy - Writing audio in output_video_with_audioTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/path/output_video_with_audio.mp4



Moviepy - Done !
Moviepy - video ready /content/path/output_video_with_audio.mp4


In [ ]:
from IPython.display import HTML, Video

video_path = "/content/output_video_with_audioTEMP_MPY_wvf_snd.mp4" #@param {type:"string"}
Video(video_path, width=400, height=300, embed=True)


Audio Generation,
Enhancing Audio Generation Using Video Clips based on Action and Emotion in Objects

In [ ]:
!nvidia-smi

In [ ]:
!pip install --quiet --upgrade git+https://github.com/huggingface/diffusers.git git+https://github.com/huggingface/transformers.git accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from diffusers import AudioLDM2Pipeline
import torch

repo_id = "cvssp/audioldm2-music"
pipe = AudioLDM2Pipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")


Loading pipeline components...:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
prompt = "Ducks in lake, Nature Sounds" #@param {type:"string"}
audio = pipe(prompt, num_inference_steps=200, audio_length_in_s=10.0).audios[0]

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
import scipy

scipy.io.wavfile.write("techno2.wav", rate=16000, data=audio)


In [ ]:
from IPython.display import Audio

Audio(audio, rate=16000)
